In [121]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'

input_file = '20190120-account-statement.xlsx'
output_file = 'output.xlsx'
currency = 'EUR'

df_raw = pd.read_excel(input_file)
df_raw.head()

,Transaction ID,Date,Details,Turnover,Balance,Currency
0,245957791,2018-02-01 23:30:00,Interest income Loan ID: 2115903-01,0.088611,0.088611,EUR
1,249313646,2018-02-08 23:30:00,Interest income Loan ID: 2126706-01,0.003458,0.092070,EUR
2,249325830,2018-02-08 23:30:00,Interest income Loan ID: 2126275-01,0.003461,0.095530,EUR
3,250118417,2018-02-09 23:30:00,Interest income Loan ID: 2126706-01,0.000865,0.096395,EUR
4,250141433,2018-02-09 23:30:00,Interest income Loan ID: 2126275-01,0.000865,0.097259,EUR


In [122]:
df = df_raw[['Date','Turnover']]
for i in range(len(df['Date'])):
    df['Date'][i] = df['Date'][i].split(' ')[0]
df.head()

,Date,Turnover
0,2018-02-01,0.088611
1,2018-02-08,0.003458
2,2018-02-08,0.003461
3,2018-02-09,0.000865
4,2018-02-09,0.000865


In [123]:
import urllib.request
import json
import datetime

def get_mid_course( date_str ):
    date = datetime.datetime.strptime(date_str, '%Y-%m-%d') - datetime.timedelta(days = 1)
    date_to_url = date.strftime('%Y-%m-%d')
    try:        
        url ='http://api.nbp.pl/api/exchangerates/rates/A/'+currency+'/'+ date_to_url +'/'
        wp = urllib.request.urlopen(url)
        pw = wp.read().decode('UTF-8')
        js = json.loads(pw)
    
        return js['rates'][0]['mid']
    
    except:
        return get_mid_course( date_to_url )


In [124]:
df1 = df.assign(mid_rate=None)
df1.head()

,Date,Turnover,mid_rate
0,2018-02-01,0.088611,None
1,2018-02-08,0.003458,None
2,2018-02-08,0.003461,None
3,2018-02-09,0.000865,None
4,2018-02-09,0.000865,None


In [125]:
for i in range(len(df1['Date'])):
    df1['mid_rate'][i] = get_mid_course(df['Date'][i])
    
df1.head()

,Date,Turnover,mid_rate
0,2018-02-01,0.088611,4.1488
1,2018-02-08,0.003458,4.1593
2,2018-02-08,0.003461,4.1593
3,2018-02-09,0.000865,4.1733
4,2018-02-09,0.000865,4.1733


In [126]:
writer = pd.ExcelWriter(output_file)
df1.to_excel(writer,'Sheet1')
writer.save()